#### Example Socrata Module for City of Chicago Data. https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi
* I'll move the client info to a config file later. 
* I don't use the sodapy package, the code is more efficient, but I'm trying to demonstrate that I'm learning the DS_Bootcamp code

In [1]:
#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import warnings;
warnings.filterwarnings('ignore');

# import pandas as pd
from sodapy import Socrata

MyAppToken = 'PoLZLhceSUgObx1Disd6WIQS1'

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofchicago.org", None)

# Example authenticated client (needed for non-public datasets):
client = Socrata('data.cityofchicago.org',
                 MyAppToken,
                 username="stephen.monteiro@northwestern.edu",
                 password="FackeTestPw0rd")

#### Importing Modules & Data
* With more time and a better coder, you could loop through each API call, hence why I define year as a list. Optimistic I'll return to it one day. 

In [2]:
#Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pprint as pp
import os
import csv
year = [2002, 2010, 2018]


In [3]:
# fbi_codex Needed to index Crime Reports into summary categories
fbi_codex = pd.read_csv('crime_resources/fbi_codex.csv')
fbi_index = fbi_codex[['fbi_Crime_type','fbi_code_key','ds_fbi_group']]
fbi_index = fbi_index.dropna()

# gent_codex Needed to index Crime Reports into summary categories
gent_codex = pd.read_csv('../Resources/UIC_gentrification_index_neighborhood_list_final.csv')
gent_concat = gent_codex["Type"].str.split(" - ", n = 1, expand = True) 
gent_codex["Classification"] = gent_concat[1]
gent_codex["Class_Code"] = gent_concat[0]
gent_fin = gent_codex.rename(columns={"No.": "community_area"})
cca_codex = pd.read_csv('../Resources/ReferenceCCA20102014.csv')
display(fbi_index.head())
display(gent_fin.head())

,fbi_Crime_type,fbi_code_key,ds_fbi_group
0,Aggravated Assault,04A,Violent
1,Aggravated Battery,04B,Violent
2,Arson,9,Nonviolent
3,Burglary,5,Nonviolent
4,Criminal Sexual Abuse,17,Nonviolent


,community_area,Neighborhood,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,77,Edgewater,7,9,9,7,9,2,No Change,1 - Upper Class,Upper Class,1
1,9,Edison Park,10,9,11,11,11,1,No Change,1 - Upper Class,Upper Class,1
2,12,Forest Glen,11,11,11,11,9,-2,No Change,1 - Upper Class,Upper Class,1
3,4,Lincoln Square,9,7,5,11,11,2,No Change,1 - Upper Class,Upper Class,1
4,13,North Park,9,11,11,11,11,2,No Change,1 - Upper Class,Upper Class,1


#### API Call for Start Year
* Currently using 2002 rather than 2001, because 2001 data lacks 'community_area' for most reported crimes. With more time, you could use loops and conditions on 'latitude' and 'longitude' to fill 'community area'.
* Export the data to CSV so that there is a reference file in case original data source is ever corrupted, or non-responsive to API calls.
* For more on the City of Chicago Crime Data: https://dev.socrata.com/foundry/data.cityofchicago.org/6zsd-86xi

* FIGURE OUT: how to remove decimal from neighborhood

In [4]:
results = client.get("6zsd-86xi", year=2002, limit=200000)
results_df = pd.DataFrame.from_records(results) # Convert to pandas DataFrame # results_df.head()
start_results = results_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
start_results['community_area'] = pd.to_numeric(start_results['community_area'], errors='coerce')
start_format = start_results.sort_values('community_area')
start_format.to_csv('report_reference_data/start_tfile.csv')    
start_format.head()


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
181967,0.0,26,2002-07-05T10:30:00.000,TELEPHONE THREAT,41.798886497,-87.790543844,OTHER OFFENSE,2002
179310,0.0,14,2002-07-09T10:00:00.000,TO VEHICLE,41.693049104,-87.690994029,CRIMINAL DAMAGE,2002
30935,0.0,07,2002-02-04T19:30:00.000,AUTOMOBILE,41.881536805,-87.616595359,MOTOR VEHICLE THEFT,2002
198253,1.0,11,2002-05-09T00:00:00.000,CREDIT CARD FRAUD,42.001185406,-87.660730781,DECEPTIVE PRACTICE,2002
124280,1.0,06,2002-05-11T13:00:00.000,$500 AND UNDER,42.001577409,-87.657389739,THEFT,2002


#### Combining the Crime Data with our Research Indexes (Gentrification & FBI Crime Category)

In [5]:
baseyear_fbiindex = start_format.merge(fbi_index, left_on='fbi_code', right_on='fbi_code_key')
b_crimecat_cca = baseyear_fbiindex.merge(gent_fin, left_on='community_area', right_on='community_area')
b_crimecat_cca.to_csv('report_reference_data/b_crimecat_cca.csv') # display(b_crimecat_cca.columns)
b_crimecat_cca.head()

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,1.0,26,2002-06-06T16:00:00.000,UNLAWFUL INTERFERE/VISITATION,42.013787071,-87.673083486,KIDNAPPING,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
1,1.0,26,2002-04-29T11:30:00.000,TO LAND,42.019383131,-87.672723831,CRIMINAL TRESPASS,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
2,1.0,26,2002-05-01T10:30:00.000,OTHER VEHICLE OFFENSE,42.004379922,-87.674501603,OTHER OFFENSE,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
3,1.0,26,2002-06-10T12:15:00.000,TO LAND,42.021723171,-87.673071312,CRIMINAL TRESPASS,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7
4,1.0,26,2002-05-12T21:00:00.000,HARASSMENT BY TELEPHONE,42.0092045,-87.660830285,OTHER OFFENSE,2002,Misc Non-Index Offense,26,...,9,9,3,1,3,-6,Negative,7 - Mild Decline,Mild Decline,7


In [6]:
b_crimecat_cca_clean = b_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
b_crimecat_cca_group = b_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
crime_reset = b_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,7123,5443,185
Gentrification,5980,3163,855
Middle Class,3460,1888,327
Mild Decline,1835,1068,147
Moderate Decline,4434,3068,146
Not Gentrification,1052,687,84
Poverty,1471,977,40
Serious Decline,9119,6360,365
Upper Class,856,468,79


In [7]:
b_crimecat_cca_clean['Class_Code'] = pd.to_numeric(b_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ngb1 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] < 3]
ngb2 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 5]
gb6 = b_crimecat_cca_clean.loc[b_crimecat_cca_clean['Class_Code'] == 6]
combo = [ngb1, ngb2]
nGentrified = pd.concat(combo)
nGentrified["DS_Subset"] = "Not Gentrified"
gb6["DS_Subset"] = "Gentrified"
neighborhodtype_bycrime = [nGentrified, gb6]
b_neighbors = pd.concat(neighborhodtype_bycrime)
b_neighbors_reset = b_neighbors.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).count()
b_neighbors_dropem = b_neighbors_reset.reset_index()
b_neighbors_dropem.drop(columns=['Classification','community_area','fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
b_neighbors_final = b_neighbors_dropem.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).sum().unstack('ds_fbi_group')
base_year_ngdata = b_neighbors_final.reset_index()# .pivot(index='Neighborhood', columns='ds_fbi_group', values='Class_Code')
base_year_ngdata.to_csv('report_reference_data/base_year_ngdata.csv')
base_year_ngdata


Count of FBI Crime Categories by Gentrification Pathway


DS_Subset     Neighborhood Class_Code                     
ds_fbi_group                                  Nonviolent Violent White Collar
0                 Gentrified        Lake View        582     271          109
1                 Gentrified     Lincoln Park        416     173           78
2                 Gentrified             Loop        430     226          158
3                 Gentrified  Near North Side       1102     602          203
4                 Gentrified  Near South Side        220     152           28
5                 Gentrified   Near West Side       1048     613           67
6                 Gentrified     North Center        245     126           20
7                 Gentrified           Uptown        687     384          106
8                 Gentrified        West Town       1250     616           86
9             Not Gentrified    Armour Square        121      43            4
10            Not Gentrified          Beverly        136      70           24
11            Not Gentrified       Bridgeport        242     170           11
12            Not Gentrified  Calumet Heights        159     100            8
13            Not Gentrified         Clearing        164      83           24
14            Not Gentrified          Dunning        257     132           18
15            Not Gentrified        Edgewater        405     225           39
16            Not Gentrified      Edison Park         35      17            1
17            Not Gentrified      Forest Glen         72      26            5
18            Not Gentrified   Garfield Ridge        370     164           55
19            Not Gentrified        Hyde Park        194      98           30
20            Not Gentrified      Irving Park        425     235           26
21            Not Gentrified   Jefferson Park        151      73           19
22            Not Gentrified          Kenwood        169     179           12
23            Not Gentrified   Lincoln Square        245     158           26
24            Not Gentrified     Logan Square        689     474           69
25            Not Gentrified      Morgan Park        322     197           19
26            Not Gentrified  Mount Greenwood         96      58            9
27            Not Gentrified       North Park         99      42            8
28            Not Gentrified     Norwood Park        178      96           14
29            Not Gentrified     Portage Park        501     202           32
30            Not Gentrified       West Ridge        338     201           37

In [8]:
midresults = client.get("6zsd-86xi", year=2010, limit=200000)
midresults_df = pd.DataFrame.from_records(midresults) # Convert to pandas DataFrame # results_df.head()
mid_results = midresults_df[['community_area','fbi_code','date','description','latitude','longitude','primary_type','year']]
mid_results['community_area'] = pd.to_numeric(mid_results['community_area'], errors='coerce')
mid_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
mid_results.sort_values(['community_area'])
mid_results.to_csv('report_reference_data/mid_tfile.csv')    
mid_results.head()

,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
0,71.0,02,2010-01-01T00:01:00.000,PREDATORY,NaN,NaN,CRIM SEXUAL ASSAULT,2010
1,41.0,11,2010-01-01T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010
2,11.0,02,2010-01-01T00:00:00.000,SEX ASSLT OF CHILD BY FAM MBR,NaN,NaN,OFFENSE INVOLVING CHILDREN,2010
3,67.0,11,2010-01-11T18:20:00.000,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,NaN,DECEPTIVE PRACTICE,2010
4,66.0,02,2010-05-01T00:00:00.000,AGGRAVATED: OTHER,NaN,NaN,CRIM SEXUAL ASSAULT,2010


In [9]:
midyear_fbiindex = mid_results.merge(fbi_index, left_on='fbi_code', right_on='fbi_code_key')
mid_crimecat_cca = midyear_fbiindex.merge(gent_fin, left_on='community_area', right_on='community_area')
mid_crimecat_cca.to_csv('report_reference_data/mid_crimecat_cca.csv') # display(b_crimecat_cca.columns)
display(mid_crimecat_cca.head())

mid_crimecat_cca_clean = mid_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
mid_crimecat_cca_group = mid_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
mid_crime_reset = mid_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
mid_crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
mid_crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')




,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,41.0,11,2010-01-01T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
1,41.0,11,2010-04-28T12:00:00.000,FINANCIAL IDENTITY THEFT $300 AND UNDER,NaN,NaN,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
2,41.0,11,2010-02-17T11:15:00.000,FINANCIAL IDENTITY THEFT OVER $ 300,NaN,NaN,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
3,41.0,11,2010-07-15T12:00:00.000,FRAUD OR CONFIDENCE GAME,41.802505287,-87.585663616,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2
4,41.0,11,2010-07-18T00:00:00.000,FRAUD OR CONFIDENCE GAME,41.793300208,-87.594025302,DECEPTIVE PRACTICE,2010,Fraud,11,...,9,9,11,7,7,-2,No Change,2 - Middle Class,Middle Class,2


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,15949,9180,747
Gentrification,8825,4895,1837
Middle Class,6800,3513,650
Mild Decline,3584,1769,277
Moderate Decline,10320,5557,502
Not Gentrification,2051,1097,167
Poverty,2704,1547,101
Serious Decline,21060,11230,1135
Upper Class,1489,807,173


In [14]:
mid_crimecat_cca_clean['Class_Code'] = pd.to_numeric(mid_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ngm1 = mid_crimecat_cca_clean.loc[mid_crimecat_cca_clean['Class_Code'] < 3]
ngm2 = mid_crimecat_cca_clean.loc[mid_crimecat_cca_clean['Class_Code'] == 5]
gm6 = mid_crimecat_cca_clean.loc[mid_crimecat_cca_clean['Class_Code'] == 6]
combo = [ngm1, ngm2]
nGentrifiedm = pd.concat(combo)
nGentrifiedm["DS_Subset"] = "Not Gentrified"
gm6["DS_Subset"] = "Gentrified"
mid_neighborhodtype_bycrime = [nGentrifiedm, gm6]
mid_neighbors = pd.concat(mid_neighborhodtype_bycrime)
mid_neighbors_reset = mid_neighbors.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).count()
mid_neighbors_dropem = mid_neighbors_reset.reset_index()
mid_neighbors_dropem.drop(columns=['Classification','community_area','fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
mid_neighbors_final = mid_neighbors_dropem.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).sum().unstack('ds_fbi_group')
mid_year_ngdata = mid_neighbors_final.reset_index()# .pivot(index='Neighborhood', columns='ds_fbi_group', values='Class_Code')
mid_year_ngdata.rename(index=str, columns={"Nonviolent": "Nonviolent_2002", "Violent": "Violent_2002", "White_Collar": "White_Collar_2002"})
mid_year_ngdata.to_csv('report_reference_data/mid_year_ngdata.csv')
mid_year_ngdata


Count of FBI Crime Categories by Gentrification Pathway


DS_Subset     Neighborhood Class_Code                     
ds_fbi_group                                  Nonviolent Violent White Collar
0                 Gentrified        Lake View        931     508          296
1                 Gentrified     Lincoln Park        665     294          152
2                 Gentrified             Loop        850     436          365
3                 Gentrified  Near North Side       1579     945          433
4                 Gentrified  Near South Side        315     189           45
5                 Gentrified   Near West Side       1549     879          165
6                 Gentrified     North Center        332     127           49
7                 Gentrified           Uptown        930     544          123
8                 Gentrified        West Town       1674     973          209
9             Not Gentrified    Armour Square        250     106           28
10            Not Gentrified          Beverly        224     104           40
11            Not Gentrified       Bridgeport        522     242           33
12            Not Gentrified  Calumet Heights        352     215           24
13            Not Gentrified         Clearing        314     151           18
14            Not Gentrified          Dunning        471     229           54
15            Not Gentrified        Edgewater        610     408           81
16            Not Gentrified      Edison Park         54      37            7
17            Not Gentrified      Forest Glen        130      52           15
18            Not Gentrified   Garfield Ridge        625     225           67
19            Not Gentrified        Hyde Park        312     194           47
20            Not Gentrified      Irving Park        801     430           63
21            Not Gentrified   Jefferson Park        324     148           44
22            Not Gentrified          Kenwood        399     299           28
23            Not Gentrified   Lincoln Square        481     221           55
24            Not Gentrified     Logan Square       1279     749          106
25            Not Gentrified      Morgan Park        658     331           46
26            Not Gentrified  Mount Greenwood        202      97           25
27            Not Gentrified       North Park        214      89           15
28            Not Gentrified     Norwood Park        242     133           35
29            Not Gentrified     Portage Park        908     472           89
30            Not Gentrified       West Ridge        968     485           70

In [11]:
closeresults = client.get("6zsd-86xi", year=2018, limit=200000)
closeresults_df = pd.DataFrame.from_records(closeresults)                    # Convert to pandas DataFrame # results_df.head()
close_results = closeresults_df[['community_area','fbi_code','date','description', 'latitude','longitude','primary_type','year']]
close_results['community_area'] = pd.to_numeric(close_results['community_area'], errors='coerce')
close_results.groupby(by=['fbi_code']).count() #relevant_results.sort_values('fbi_code')
close_results.to_csv('report_reference_data/close_tfile.csv')    
close_results.head()


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year
0,53.0,01A,2018-02-04T01:36:00.000,FIRST DEGREE MURDER,41.68073915,-87.645661144,HOMICIDE,2018
1,75.0,08B,2018-01-07T21:01:00.000,DOMESTIC BATTERY SIMPLE,41.692258729,-87.65053259,BATTERY,2018
2,8.0,06,2018-01-05T18:10:00.000,RETAIL THEFT,41.890384236,-87.624109917,THEFT,2018
3,29.0,18,2018-01-08T08:59:00.000,MANU/DELIVER: HEROIN (WHITE),41.858283017,-87.723691271,NARCOTICS,2018
4,39.0,26,2018-01-08T10:45:00.000,HARASSMENT BY TELEPHONE,41.809297059,-87.591967028,OTHER OFFENSE,2018


In [12]:
closeyear_fbiindex = close_results.merge(fbi_index, left_on='fbi_code', right_on='fbi_code_key')
close_crimecat_cca = closeyear_fbiindex.merge(gent_fin, left_on='community_area', right_on='community_area')
close_crimecat_cca.to_csv('report_reference_data/close_crimecat_cca.csv') # display(b_crimecat_cca.columns)
display(close_crimecat_cca.head())

close_crimecat_cca_clean = close_crimecat_cca[['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type', 'ds_fbi_group', 'Classification', 'Class_Code',]]
close_crimecat_cca_group = close_crimecat_cca_clean.groupby(by=['ds_fbi_group','Classification']).count() #relevant_results.sort_values('fbi_code')
close_crime_reset = close_crimecat_cca_group.reset_index()
print("Count of FBI Crime Categories by Gentrification Pathway")
close_crime_reset.drop(columns=['community_area', 'Neighborhood', 'fbi_code','fbi_Crime_type',])
close_crime_reset.pivot(index='Classification', columns='ds_fbi_group', values='Class_Code')


,community_area,fbi_code,date,description,latitude,longitude,primary_type,year,fbi_Crime_type,fbi_code_key,...,1970,1980,1990,2000,2010,1970-2010,Change,Type,Classification,Class_Code
0,75.0,08B,2018-01-07T21:01:00.000,DOMESTIC BATTERY SIMPLE,41.692258729,-87.65053259,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
1,75.0,08B,2018-02-11T20:30:00.000,SIMPLE,41.694700209,-87.682526799,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
2,75.0,08B,2018-03-03T23:30:00.000,DOMESTIC BATTERY SIMPLE,41.693188884,-87.64819774,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
3,75.0,08B,2018-04-23T11:41:00.000,DOMESTIC BATTERY SIMPLE,41.693258507,-87.642408263,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2
4,75.0,08B,2018-04-16T01:00:00.000,SIMPLE,41.694080673,-87.650945376,BATTERY,2018,Simple Battery,08B,...,5,3,3,3,1,-4,No Change,2 - Middle Class,Middle Class,2


Count of FBI Crime Categories by Gentrification Pathway


ds_fbi_group,Nonviolent,Violent,White Collar
Classification,,,
Extreme Poverty,13641,9745,1239
Gentrification,7544,6583,4928
Middle Class,5201,3811,1394
Mild Decline,2482,1968,584
Moderate Decline,7563,5251,930
Not Gentrification,1285,999,381
Poverty,2039,1713,305
Serious Decline,14721,12234,2162
Upper Class,1164,952,441


In [13]:
close_crimecat_cca_clean['Class_Code'] = pd.to_numeric(close_crimecat_cca_clean['Class_Code'], errors='coerce') # b_crimecat_cca_clean.dtypes
ngc1 = close_crimecat_cca_clean.loc[close_crimecat_cca_clean['Class_Code'] < 3]
ngc2 = close_crimecat_cca_clean.loc[close_crimecat_cca_clean['Class_Code'] == 5]
gc6 = close_crimecat_cca_clean.loc[close_crimecat_cca_clean['Class_Code'] == 6]
combo = [ngc1, ngc2]
nGentrifiedc = pd.concat(combo)
nGentrifiedc["DS_Subset"] = "Not Gentrified"
gc6["DS_Subset"] = "Gentrified"
close_neighborhodtype_bycrime = [nGentrifiedc, gc6]
close_neighbors = pd.concat(close_neighborhodtype_bycrime)
close_neighbors_reset = close_neighbors.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).count()
close_neighbors_dropem = close_neighbors_reset.reset_index()
close_neighbors_dropem.drop(columns=['Classification','community_area','fbi_code','fbi_Crime_type',], inplace=True)
print("Count of FBI Crime Categories by Gentrification Pathway")
close_neighbors_final = close_neighbors_dropem.groupby(by=['DS_Subset','Neighborhood','ds_fbi_group']).sum().unstack('ds_fbi_group')
close_year_ngdata = close_neighbors_final.reset_index()# .pivot(index='Neighborhood', columns='ds_fbi_group', values='Class_Code')
close_year_ngdata.to_csv('report_reference_data/close_year_ngdata.csv')
close_year_ngdata


Count of FBI Crime Categories by Gentrification Pathway


DS_Subset     Neighborhood Class_Code                     
ds_fbi_group                                  Nonviolent Violent White Collar
0                 Gentrified        Lake View        820     742          501
1                 Gentrified     Lincoln Park        632     368          336
2                 Gentrified             Loop       1050    1015         1164
3                 Gentrified  Near North Side       1351    1383         1492
4                 Gentrified  Near South Side        335     261          171
5                 Gentrified   Near West Side       1449    1178          548
6                 Gentrified     North Center        208     135           94
7                 Gentrified           Uptown        597     666          251
8                 Gentrified        West Town       1102     835          371
9             Not Gentrified    Armour Square        153     158           53
10            Not Gentrified          Beverly        203     120           61
11            Not Gentrified       Bridgeport        345     218           88
12            Not Gentrified  Calumet Heights        273     263           69
13            Not Gentrified         Clearing        225     155           62
14            Not Gentrified          Dunning        314     258           82
15            Not Gentrified        Edgewater        478     438          180
16            Not Gentrified      Edison Park         59      51           19
17            Not Gentrified      Forest Glen         88      45           43
18            Not Gentrified   Garfield Ridge        539     262          128
19            Not Gentrified        Hyde Park        353     183          133
20            Not Gentrified      Irving Park        522     435          126
21            Not Gentrified   Jefferson Park        242     167           68
22            Not Gentrified          Kenwood        303     226           80
23            Not Gentrified   Lincoln Square        372     303          143
24            Not Gentrified     Logan Square        787     623          240
25            Not Gentrified      Morgan Park        485     382          100
26            Not Gentrified  Mount Greenwood        139      78           48
27            Not Gentrified       North Park        167     115           56
28            Not Gentrified     Norwood Park        252     187           87
29            Not Gentrified     Portage Park        661     528          168
30            Not Gentrified       West Ridge        690     567          182